In [1]:
import pandas as pd
import numpy as np
import os
import scipy
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
# Load Neural Data

# First load test indices
df = pd.read_csv('/scratch/soroush1/memorability/muri1320/test.csv')
test_indices = df.values.flatten()
test_indices.shape

# load neural data
# How to read the .h5 file
import h5py

neural_type = "active"

neural_data_path = f"/scratch/soroush1/idiosyncrasy/neural_data/rates_magneto_{neural_type}.h5"
monkey_name = f"magneto/{neural_type}"
monkey_data = None

with h5py.File(neural_data_path, 'r') as f:
    print(f"{list(f.keys()) = }")
    # Print structure of the data
    def print_structure(name, obj):
        print(name, type(obj))
        if isinstance(obj, h5py.Group):
            for key in obj.keys():
                print(f"  {key}")
    
    f.visititems(print_structure)
    
    # Get the data
    monkey_data = f[monkey_name][:]
    time, images, neural_sites, reps = monkey_data.shape
    print("Dataset shape:", monkey_data.shape) # time images neural-sites reps

# load train and test indices
train_neural_indices = np.load('meta/train_neural_indices.npy')
test_neural_indices = np.load('meta/test_neural_indices.npy')

# select only the test indices & test neural data
neural_data = monkey_data[:, test_indices, :, :]
neural_data = neural_data[:, :, test_neural_indices, :]

# select only the time between 70 to 170 ms
neural_data = np.nanmean(neural_data[7:17, :, :, :], axis=0)

assert neural_data.shape == (660, 144, 70) # 660 images, 144 neural sites, 70 reps

list(f.keys()) = ['magneto']
magneto <class 'h5py._hl.group.Group'>
  active
magneto/active <class 'h5py._hl.dataset.Dataset'>
Dataset shape: (26, 1320, 288, 70)


In [3]:
# Load model Feature
# Verify saved features
output_path = "model_features/raw_resnet50_features.h5"

with h5py.File(output_path, 'r') as f:
    print("Saved features shape:", f['features'].shape)
    print("Saved labels shape:", f['labels'].shape)

    model_features = f['features'][:]

print(f"{model_features.shape}")

Saved features shape: (660, 65536)
Saved labels shape: (660,)
(660, 65536)


In [4]:
import math
import torch
import numpy as np

class CKA(object):
    def __init__(self):
        pass 
    
    def centering(self, K):
        n = K.shape[0]
        unit = np.ones([n, n])
        I = np.eye(n)
        H = I - unit / n
        return np.dot(np.dot(H, K), H) 

    def rbf(self, X, sigma=None):
        GX = np.dot(X, X.T)
        KX = np.diag(GX) - GX + (np.diag(GX) - GX).T
        if sigma is None:
            mdist = np.median(KX[KX != 0])
            sigma = math.sqrt(mdist)
        KX *= - 0.5 / (sigma * sigma)
        KX = np.exp(KX)
        return KX
 
    def kernel_HSIC(self, X, Y, sigma):
        return np.sum(self.centering(self.rbf(X, sigma)) * self.centering(self.rbf(Y, sigma)))

    def linear_HSIC(self, X, Y):
        L_X = X @ X.T
        L_Y = Y @ Y.T
        return np.sum(self.centering(L_X) * self.centering(L_Y))

    def linear_CKA(self, X, Y):
        hsic = self.linear_HSIC(X, Y)
        var1 = np.sqrt(self.linear_HSIC(X, X))
        var2 = np.sqrt(self.linear_HSIC(Y, Y))

        return hsic / (var1 * var2)

    def kernel_CKA(self, X, Y, sigma=None):
        hsic = self.kernel_HSIC(X, Y, sigma)
        var1 = np.sqrt(self.kernel_HSIC(X, X, sigma))
        var2 = np.sqrt(self.kernel_HSIC(Y, Y, sigma))

        return hsic / (var1 * var2)

In [5]:
neural_data.shape

(660, 144, 70)

In [6]:
cka = CKA()
cka_similarity = cka.linear_CKA(model_features, np.nanmean(neural_data, axis=2))

cka_similarity

0.41510949557566984